In [4]:
#Importing dependencies
import pandas as pd
import numpy as np
#import tensorflow as tf
import random
import os
import matplotlib.pyplot as plt
from datetime import datetime as dt
from sklearn.datasets import load_iris
from sklearn.utils import shuffle
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import roc_auc_score,log_loss,accuracy_score
from sklearn.model_selection import KFold
from sklearn.model_selection import train_test_split
from tqdm import tqdm,tqdm_notebook,_tqdm_notebook
tqdm.pandas()
%matplotlib inline

import tensorflow.compat.v1 as tf
tf.disable_v2_behavior() 

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:15: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.*` instead of `tqdm._tqdm_notebook.*`
  from ipykernel import kernelapp as app


Instructions for updating:
non-resource variables are not supported in the long term


In [6]:
#Prepping the data
url='https://raw.githubusercontent.com/254samo/SolvedBI/master/On_Time_Performance.csv'
df = pd.read_csv(url)

df = df[df['Facility']=='JFK']
df = df.reset_index()
del df['index']

df['Date'] = pd.to_datetime(df['Date'], errors='coerce')
#df.head()

df['month'] = df['Date'].dt.month
df['weekday'] = df['Date'].dt.day_name()
#df.head()

df['80% or More On Time'] = 0
df.loc[df['% On-Time Gate Arrivals'] >= 80, '80% or More On Time'] = 1

df['month'] = df['month'].apply(str)
df = df.drop(['Facility'], axis=1)
#df.head()

Extracting the Peak Hours

In [7]:
#Extracting the Peak Hours 
peakHours = df.loc[((df['Hour']>=6) & (df['Hour']<= 9)) | ((df['Hour'] >= 15) & (df['Hour']<= 18)) | ((df['Hour']>= 20)) | ((df['Hour']<= 1))]
peakHours = peakHours.drop(['Average Gate Arrival Delay', 'Scheduled Departures', 'Scheduled Arrivals', 'Date', '% On-Time Gate Arrivals', 'Average Gate Departure Delay', 'Hour', 'month', 'weekday'], axis=1)


Extracting the Off-Peak Hours

In [8]:
#Extracting the Off-Peak Hours 
offPeak = df.loc[((df['Hour']>=2) & (df['Hour']<= 5)) | ((df['Hour'] >= 10) & (df['Hour']<= 14)) | ((df['Hour']== 19))]
offPeak = offPeak.drop(['Average Gate Arrival Delay', 'Scheduled Departures', 'Scheduled Arrivals', 'Date', '% On-Time Gate Arrivals', 'Average Gate Departure Delay', 'Hour', 'month', 'weekday'], axis=1)

offPeak.head(10)

,Departures For Metric Computation,Arrivals For Metric Computation,% On-Time Gate Departures,% On-Time Airport Departures,Average Taxi Out Time,Average Taxi Out Delay,Average Airport Departure Delay,Average Airborne Delay,Average Taxi In Delay,Average Block Delay,80% or More On Time
2,5.0,1.0,100.00,100.00,16.00,1.08,1.00,0.00,6.70,2.00,1
3,0.0,2.0,0.00,0.00,0.00,0.00,0.00,6.00,5.70,3.00,0
4,2.0,0.0,100.00,100.00,16.50,1.20,1.00,0.00,0.00,0.00,0
5,8.0,21.0,87.50,87.50,22.50,6.00,22.50,5.33,4.92,4.81,1
10,22.0,25.0,81.82,77.27,28.86,11.52,34.14,2.20,1.84,1.96,1
11,33.0,18.0,84.85,84.85,21.61,4.21,9.70,1.06,3.23,0.44,1
12,23.0,31.0,78.26,78.26,19.91,3.75,39.26,1.35,2.17,1.29,1
13,24.0,40.0,91.67,83.33,23.00,5.85,8.79,2.17,2.93,3.08,1
14,30.0,41.0,70.00,56.67,23.43,6.23,34.80,3.02,2.79,1.54,1
19,40.0,28.0,57.50,35.00,31.35,13.22,32.78,3.25,10.30,8.68,0


In [9]:
# To pass parameters to the MLP, enter the the classification feature's name to the "the_class_column"
the_class_column = '80% or More On Time'
#Specify a desired Loss Rate
Loss_Rate = 0.0003 #0.0003
# Specify the epoch
Epochs = 50
# Specify the Batch Size
BatchSize = 4
# Specify the number of neurons for Hidden Layer one and Hidden Layer two
NoOfHiddenNeuronsLayer1 = 32
NoOfHiddenNeuronsLayer2 = 16

In [10]:
#!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!
#                       DO NOT EDIT THIS CELL
#!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!
def perceptron_prep_and_implementation(filename):
    data = filename
    
    
    #Suppressing warnings and infos of TensorFlow
    os.environ['TF_CPP_MIN_LOG_LEVEL'] = '3' 
    SEED = 1000 #Fixing seed
    synthetic = False# whether to use synthetic data

    def seed_everything(seed=SEED):
	    random.seed(seed)
	    os.environ['PYTHONHASHSEED'] = str(seed)
	    np.random.seed(seed)
	    tf.set_random_seed(seed) #tf.random.set_seed(seed)
    seed_everything(seed=SEED)

	# THIS MARKS THE BEGINNING OF OUR PERCEPTION PREP
	#We assign the dataframe to be processed to the dataframe name that is used in the perceptron
	
	

    # HyperParameters
    lr = Loss_Rate
    training_epochs = Epochs
    batch_size = BatchSize
    pred_size = int(batch_size * 2)


	
	
	#Shuffling data and resetting index
    data = shuffle(data)
    data = data.reset_index(drop=True)
	
	#Now since data preparation is done, we assign the index to the class_column variable to a parameter
    the_class_column_index = data.columns.get_loc(the_class_column)

    # Get the Count of Rows and Columns in data
    total_rows_in_data = int(len(data.axes[0]))
    total_cols_in_data = int(len(data.axes[1]))

    #Separating input and output
    X = data.iloc[:, :total_cols_in_data]
    X.drop(X.columns[the_class_column_index], axis=1, inplace=True )
    y = data.iloc[:, the_class_column_index]

    # Get the Count of Columns in the shuffled X and y
    total_cols_in_X = int(len(X.axes[1]))
    total_cols_in_y = int(1)

    # Get the Count of Records in the shuffled X and y
    total_rows_in_X = int(len(X.axes[0]))
    total_rows_in_y = int(len(y.axes[0]))

    if synthetic:
        X = np.random.randn(total_rows_in_X,total_cols_in_X)
        y = np.random.randn(total_rows_in_y,total_cols_in_y)

        #  y = np.where(y[:,0]<0,1)
        X,y = shuffle(X,y)

    
    # Get the Count of Columns in the shuffled X and y
    total_cols_in_X = int(len(X.axes[1]))
    total_cols_in_y = int(1)

    # Get the Count of Records in the shuffled X and y
    total_rows_in_X = int(len(X.axes[0]))
    total_rows_in_y = int(len(y.axes[0]))

    print(X.shape[0],y.shape[0])
    #Train data
    train_X = X[:int(total_rows_in_X*0.8)]
    train_y = y[:int(total_rows_in_y*0.8)]
    #Test data
    test_X = X[int(total_rows_in_X*0.8):]
    test_y = y[int(total_rows_in_y*0.8):]

    #Scaling
    sc_X = StandardScaler()
    train_X = sc_X.fit_transform(train_X)
    test_X = sc_X.transform(test_X)

    #converting y to one hot formats
    train_y = pd.get_dummies(train_y).values
    test_y = pd.get_dummies(test_y).values

    #5 fold validation
    folds = KFold(n_splits= 5, shuffle=True, random_state=SEED)


    # Network Parameters
    n_hidden_1 = NoOfHiddenNeuronsLayer1 #number of neurons in 1st layer 
    n_hidden_2 = NoOfHiddenNeuronsLayer2 #number of neurons in 2nd layer 
    n_input = total_cols_in_X #No of columns/Features in X
    n_classes = y.nunique() #Output classes

    if synthetic:
        n_hidden_1 = 2048
        n_hidden_2 = 1024
        n_classes = 2
        batch_size = 4096
        pred_size = 8192
    steps = len(train_X) # How many training data

    #Resetting the graph
    tf.compat.v1.get_default_graph() #tf.reset_default_graph()

    #Defining Placeholders
    X = tf.placeholder("float", [None, n_input])
    Y = tf.placeholder("float", [None, n_classes])
    hold_prob1 = tf.placeholder(tf.float32)
    hold_prob2 = tf.placeholder(tf.float32)

    # Store layers weight & bias
    weights = {
        'h1': tf.Variable(tf.truncated_normal([n_input, n_hidden_1],\
            mean = 0.0,stddev=0.2)),
        'h2': tf.Variable(tf.truncated_normal([n_hidden_1, n_hidden_2],\
            mean = 0.0,stddev = 0.2)),
        'out': tf.Variable(tf.truncated_normal([n_hidden_2, n_classes],\
            mean = 0.0,stddev = 0.2))
    }
    biases = {
        'b1': tf.Variable(tf.constant(0.1,shape = [n_hidden_1])),
        'b2': tf.Variable(tf.constant(0.1,shape = [n_hidden_2])),
        'out': tf.Variable(tf.constant(0.1,shape = [n_classes]))
    }

    # Create model
    def multilayer_perceptron(x):
        # First Hidden Layer
        layer_1 = tf.add(tf.matmul(x, weights['h1']), biases['b1'])
        #Applying RELU nonlinearity
        layer1_RELU = tf.nn.relu(layer_1)
        #Applying Dropout
        layer1_dropout = tf.nn.dropout(layer1_RELU,keep_prob=hold_prob1)
        # Second hidden layer
        layer_2 = tf.add(tf.matmul(layer1_dropout, weights['h2']), biases['b2'])
        #Applying TANH nonlinearity
        layer2_TANH = tf.nn.tanh(layer_2)
        #Applying Dropout
        layer2_dropout = tf.nn.dropout(layer2_TANH,keep_prob=hold_prob2)
        # Output layer
        out_layer = tf.matmul(layer2_dropout, weights['out']) + biases['out']
        return out_layer


    # Building model
    logits = multilayer_perceptron(X)

    # Defining Loss Function
    loss_op = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits_v2(
        logits=logits, labels=Y))
    #Defining optimizer
    optimizer = tf.train.RMSPropOptimizer(learning_rate=lr)
    #Defining what to minimize
    train_op = optimizer.minimize(loss_op)
    # Initializing the variables
    init = tf.global_variables_initializer()

    #Graph equations for Accuracy
    matches = tf.equal(tf.argmax(logits,1),tf.argmax(Y,1))
    acc = tf.reduce_mean(tf.cast(matches,tf.float32))

    remaining_train = len(train_X) % batch_size
    remaining_test= len(test_X) % pred_size

    # Create arrays and dataframes to store results
    oof_preds = np.zeros(( train_X.shape[0] , n_classes ) )
    sub_preds = np.zeros(( test_X.shape[0] , n_classes ) )




    #Opening our Session
    with tf.Session() as sess:
        
        #k-fold validation
        for n_fold, (train_idx, valid_idx) in enumerate(folds.split(train_X,train_y)):
            print( "Fold {} has started".format(n_fold) )
            X_train, y_train = train_X[train_idx], train_y[train_idx]
            X_cv, y_cv = train_X[valid_idx], train_y[valid_idx]
            #Running initialization of variables
            sess.run(init)
            # Writing down the for loop for epochs
            for epoch in tqdm(range(training_epochs)):
                #For loop for batches
                for i in range(0,steps - remaining_train , batch_size):
                    #Getting training data to be fed into the graphs
                    batch_x, batch_y = X_train[i:i+batch_size],y_train[i:i+batch_size]
                    # Training batch by batch
                    _, c = sess.run([train_op, loss_op], feed_dict={X: batch_x,Y: batch_y,\
                        hold_prob1:0.7,hold_prob2:0.8})
                
                #Feeding last batch of train data
                batch_x, batch_y = X_train[-remaining_train:],y_train[-remaining_train:]
                _, c = sess.run([train_op, loss_op], feed_dict={X: batch_x,Y: batch_y,\
                    hold_prob1:0.7,hold_prob2:0.8})

            #Calculating predictions over validation data
            remaining_cv = len(X_cv) % pred_size
            preds_on_cv = list()

            for j in range(0,len(X_cv) - remaining_cv,pred_size):
                batch_x = X_cv[j:j+pred_size]
                preds_on_cv_temp = tf.nn.softmax(logits).eval(
                    feed_dict ={X: batch_x,hold_prob1:1.0, hold_prob2:1.0})
                preds_on_cv_temp = list(preds_on_cv_temp)
                preds_on_cv.extend(preds_on_cv_temp)

            #Calculating predictions over last batch of CV data
            if remaining_cv != 0:
                batch_x = X_cv[-remaining_cv:]
                preds_on_cv_temp = tf.nn.softmax(logits).eval(
                        feed_dict ={X: batch_x,hold_prob1:1.0, hold_prob2:1.0})
                preds_on_cv_temp = list(preds_on_cv_temp)
                preds_on_cv.extend(preds_on_cv_temp)

            #Calculating loss and accuracy for CV
            acc_on_cv,loss_on_cv = sess.run([acc,loss_op],feed_dict ={X: X_cv, Y: y_cv,\
                hold_prob1:1.0, hold_prob2:1.0})
            #Calculating AUC on Cross Validation data
            auc_on_cv = roc_auc_score(y_cv,preds_on_cv)
            #Printing CV statistics after each epoch
            print(" Accuracy: ",acc_on_cv,"\n Loss: ",loss_on_cv,"\n AUC: ",auc_on_cv)
            
            #Calculating predictions over test data
            preds_on_test = list()
            for j in range(0,len(test_X) - remaining_test,pred_size):
                batch_x = test_X[j:j+pred_size]
                preds_on_test_temp = tf.nn.softmax(logits).eval(
                    feed_dict ={X: batch_x,hold_prob1:1.0, hold_prob2:1.0})
                preds_on_test_temp = list(preds_on_test_temp)
                preds_on_test.extend(preds_on_test_temp)
            #Calculating predictions over last batch of test data
            if remaining_test != 0:
                batch_x = test_X[-remaining_test:]
                preds_on_test_temp = sess.run(tf.nn.softmax(logits),\
                        feed_dict ={X: batch_x,hold_prob1:1.0, hold_prob2:1.0})
                preds_on_test_temp = list(preds_on_test_temp)
                preds_on_test.extend(preds_on_test_temp)
        oof_preds[valid_idx] = np.array(preds_on_cv)
        sub_preds += np.array(preds_on_test) / folds.n_splits
        print("\n Validation Results: ")
        print(" Accuracy: ",acc_on_cv,"\n Loss: ",loss_on_cv,"\n AUC: ",auc_on_cv)
    print("\n Training and Scoring on Validation data done")


    acc_on_test = accuracy_score(test_y.argmax(axis=1),sub_preds.argmax(axis=1))
    loss_on_test = log_loss(test_y,sub_preds)
    auc_on_test = roc_auc_score(test_y,sub_preds)
    print("\n Test Results are below: ")
    print(" Accuracy: ",acc_on_test," \n Loss: ",loss_on_test," \n AUC: ",auc_on_test)

In [11]:
perceptron_prep_and_implementation(peakHours)

1288 1288
Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.
Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor
Fold 0 has started


100%|██████████| 50/50 [00:09<00:00,  5.42it/s]


 Accuracy:  0.74271846 
 Loss:  0.48532787 
 AUC:  0.8332829960736938


  2%|▏         | 1/50 [00:00<00:09,  5.42it/s]

Fold 1 has started


...

 Accuracy:  0.7815534 
 Loss:  0.5015647 
 AUC:  0.8259242957746479


  2%|▏         | 1/50 [00:00<00:09,  5.21it/s]

Fold 4 has started


100%|██████████| 50/50 [00:09<00:00,  5.45it/s]


 Accuracy:  0.776699 
 Loss:  0.45959997 
 AUC:  0.8496848739495798

 Validation Results: 
 Accuracy:  0.776699 
 Loss:  0.45959997 
 AUC:  0.8496848739495798

 Training and Scoring on Validation data done

 Test Results are below: 
 Accuracy:  0.7829457364341085  
 Loss:  0.4811073233833319  
 AUC:  0.8621434470491074


In [12]:
perceptron_prep_and_implementation(offPeak)

920 920
Fold 0 has started


100%|██████████| 50/50 [00:06<00:00,  7.42it/s]


 Accuracy:  0.6891892 
 Loss:  0.56459427 
 AUC:  0.7674632352941176


  2%|▏         | 1/50 [00:00<00:06,  7.45it/s]

Fold 1 has started


...

 Accuracy:  0.64625853 
 Loss:  0.65516 
 AUC:  0.6951492537313433


  2%|▏         | 1/50 [00:00<00:06,  7.42it/s]

Fold 4 has started


100%|██████████| 50/50 [00:06<00:00,  7.46it/s]


 Accuracy:  0.6530612 
 Loss:  0.64701027 
 AUC:  0.7358385783043317

 Validation Results: 
 Accuracy:  0.6530612 
 Loss:  0.64701027 
 AUC:  0.7358385783043317

 Training and Scoring on Validation data done

 Test Results are below: 
 Accuracy:  0.7717391304347826  
 Loss:  0.45134448875470035  
 AUC:  0.8680588942307692
